### Maximum Sharpe Ratio Portfolio

 The portfolio that offers the highest Sharpe ratio, meaning it provides the best risk-adjusted return. 

In [ ]:
from datetime import date, datetime, timedelta

import fireducks.pandas as pd # FireDucks also provides a pandas-like module
# Python API for Federal Reserve Economic Data (FRED) from St. Louis Fed
from fredapi import Fred
import numpy as np
import yfinance as yf

from datetime import date, timedelta

In [2]:
# Example stocks and the time period chosen
stock_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

end_date = date.today()
start_date = end_date - timedelta(days=365)

# We get Treasury yield data from FRED
api_key = "0d206dd5b47e2d7e6a217b6cdc7a960e"

In [ ]:
#Extraction of relevant data from yfinance
data = yf.download(
    stock_symbols,
    start=start_date, end=end_date,
    )['Close']
data

[*********************100%***********************]  5 of 5 completed

5 Failed downloads:
['MSFT', 'TSLA', 'GOOGL', 'AMZN', 'AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,


In [4]:
# Returns
returns = data.pct_change(periods=1).dropna() 
# Fractional change between the current and a prior element (by the periods).
returns

Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,


In [12]:
start_date

datetime.date(2024, 3, 18)

In [ ]:
# Annualize returns (CAGR)
total_returns = data.iloc[-1] / data.iloc[0] - 1
total_returns

IndexError: single positional indexer is out-of-bounds

In [ ]:
trading_days = returns.shape[0]
trading_days

249

In [ ]:

annualized_returns = (1 + total_returns) ** (252 / trading_days) - 1
# Expected Return Matrix
ER_matrix = annualized_returns.values.reshape(-1, 1)
# pass -1 to a unknown dimension (the value will be inferred)
print(ER_matrix)


[[ 0.32750368]
 [ 0.13299242]
 [ 0.21338847]
 [-0.05366621]
 [ 0.25300828]]


In [ ]:
#Covariance Matrix
cov_matrix = returns.cov()
cov_matrix

Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Ticker,,,,,
AAPL,0.000236,0.000093,0.000091,0.000094,0.000245
AMZN,0.000093,0.000310,0.000183,0.000157,0.000287
GOOGL,0.000091,0.000183,0.000332,0.000127,0.000282
MSFT,0.000094,0.000157,0.000127,0.000185,0.000197
TSLA,0.000245,0.000287,0.000282,0.000197,0.001789


In [ ]:
#Inverse Covariance Matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)
inv_cov_matrix

array([[ 5742.00269761,    57.60378928,  -336.05788588, -2210.11843689,
         -498.08578794],
       [   57.60378928,  6701.43926221, -1874.09097324, -4075.71451485,
         -336.83194379],
       [ -336.05788588, -1874.09097324,  4852.92870993, -1261.5429725 ,
         -279.29907561],
       [-2210.11843689, -4075.71451485, -1261.5429725 , 10890.26231882,
          -47.02350373],
       [ -498.08578794,  -336.83194379,  -279.29907561,   -47.02350373,
          730.28310165]])

In [ ]:
# Initialize FRED object
fred = Fred(api_key=api_key)

# Fetch 10-Year Treasury Constant Maturity Rate (GS10)
treasury_yield = fred.get_series('GS10', start_date, end_date)



In [ ]:
# Average risk-free rate over the period
avg_rf_rate = treasury_yield.mean()/100
print(f"The avg risk free rate applicable for the period is {avg_rf_rate:.2f}")

The avg risk free rate applicable for the period is 0.04


In [ ]:
ones_column = np.ones((len(stock_symbols), 1)) #The ones column
scaled_rf = np.dot(avg_rf_rate, ones_column) #risk free rate scaled
scaled_rf

array([[0.04275833],
       [0.04275833],
       [0.04275833],
       [0.04275833],
       [0.04275833]])

In [ ]:
x = (ER_matrix - scaled_rf)
x

array([[ 0.28474535],
       [ 0.09023408],
       [ 0.17063013],
       [-0.09642454],
       [ 0.21024995]])

In [ ]:
numerator = np.dot(inv_cov_matrix,x)
denominator = np.dot(np.dot(ones_column.T,inv_cov_matrix),x)
#Optimal Weights for Max Sharpe Ratio
wts = numerator/denominator
print(wts)
print(wts.sum())

[[ 2.78711221]
 [ 1.02750885]
 [ 1.03191697]
 [-3.74469099]
 [-0.10184705]]
0.9999999999999996


In [ ]:
weights = np.array(wts)
# Portfolio returns and standard deviation
portfolio_return = np.dot(annualized_returns, weights)
portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() *
252, weights)))
# {ortfolio Sharpe ratio
portfolio_sharpe_ratio = (portfolio_return - avg_rf_rate) / portfolio_std_dev
print(f"Portfolio Sharpe Ratio: {portfolio_sharpe_ratio}")

Portfolio Sharpe Ratio: [[1.83743721]]
